In [1]:
import esmpy
import numpy as np
import xarray as xr

In [2]:
def grid_create_from_coordinates_3d(xcoords, ycoords, zcoords, xcorners=False, ycorners=False, zcorners=False, corners=False, domask=False, doarea=False):
    """
    Create a 3 dimensional Grid using the xcoordinates, ycoordinates and zcoordinates.
    :param xcoords: The 1st dimension or 'x' coordinates at cell centers, as a Python list or numpy Array
    :param ycoords: The 2nd dimension or 'y' coordinates at cell centers, as a Python list or numpy Array
    :param zcoords: The 3rd dimension or 'z' coordinates at cell centers, as a Python list or numpy Array
    :param xcorners: The 1st dimension or 'x' coordinates at cell corners, as a Python list or numpy Array
    :param ycorners: The 2nd dimension or 'y' coordinates at cell corners, as a Python list or numpy Array
    :param zcorners: The 3rd dimension or 'z' coordinates at cell corners, as a Python list or numpy Array
    :param corners: boolean to determine whether or not to add corner coordinates to this grid
    :param domask: boolean to determine whether to set an arbitrary mask or not
    :param doarea: boolean to determine whether to set an arbitrary area values or not
    :return: grid
    """
    [x, y, z] = [0, 1, 2]

    # create a grid given the number of grid cells in each dimension, the center stagger location is allocated and the
    # Cartesian coordinate system is specified
    max_index = np.array([len(xcoords), len(ycoords), len(zcoords)])
    grid = esmpy.Grid(max_index, staggerloc=[esmpy.StaggerLoc.CENTER_VCENTER], coord_sys=esmpy.CoordSys.SPH_DEG)

    # set the grid coordinates using numpy arrays, parallel case is handled using grid bounds
    gridXCenter = grid.get_coords(x)
    x_par = xcoords[grid.lower_bounds[esmpy.StaggerLoc.CENTER_VCENTER][x]:grid.upper_bounds[esmpy.StaggerLoc.CENTER_VCENTER][x]]
    gridXCenter[...] = x_par.reshape(x_par.size, 1, 1)

    gridYCenter = grid.get_coords(y)
    y_par = ycoords[grid.lower_bounds[esmpy.StaggerLoc.CENTER_VCENTER][y]:grid.upper_bounds[esmpy.StaggerLoc.CENTER_VCENTER][y]]
    gridYCenter[...] = y_par.reshape(1, y_par.size, 1)

    gridZCenter = grid.get_coords(z)
    z_par = zcoords[grid.lower_bounds[esmpy.StaggerLoc.CENTER_VCENTER][z]:grid.upper_bounds[esmpy.StaggerLoc.CENTER_VCENTER][z]]
    gridZCenter[...] = z_par.reshape(1, 1, z_par.size)

    # create grid corners in a slightly different manner to account for the bounds format common in CF-like files
    if corners:
        grid.add_coords([esmpy.StaggerLoc.CORNER_VFACE])
        lbx = grid.lower_bounds[esmpy.StaggerLoc.CORNER_VFACE][x]
        ubx = grid.upper_bounds[esmpy.StaggerLoc.CORNER_VFACE][x]
        lby = grid.lower_bounds[esmpy.StaggerLoc.CORNER_VFACE][y]
        uby = grid.upper_bounds[esmpy.StaggerLoc.CORNER_VFACE][y]
        lbz = grid.lower_bounds[esmpy.StaggerLoc.CORNER_VFACE][z]
        ubz = grid.upper_bounds[esmpy.StaggerLoc.CORNER_VFACE][z]

        gridXCorner = grid.get_coords(x, staggerloc=esmpy.StaggerLoc.CORNER_VFACE)
        for i0 in range(ubx - lbx - 1):
            gridXCorner[i0, :, :] = xcorners[i0+lbx, 0]
        gridXCorner[i0 + 1, :, :] = xcorners[i0+lbx, 1]

        gridYCorner = grid.get_coords(y, staggerloc=esmpy.StaggerLoc.CORNER_VFACE)
        for i1 in range(uby - lby - 1):
            gridYCorner[:, i1, :] = ycorners[i1+lby, 0]
        gridYCorner[:, i1 + 1, :] = ycorners[i1+lby, 1]

        gridZCorner = grid.get_coords(z, staggerloc=esmpy.StaggerLoc.CORNER_VFACE)
        for i2 in range(ubz - lbz - 1):
            gridZCorner[:, :, i2] = zcorners[i2+lbz, 0]
        gridZCorner[:, :, i2 + 1] = zcorners[i2+lbz, 1]

    # add an arbitrary mask
    if domask:
        mask = grid.add_item(esmpy.GridItem.MASK)
        mask[:] = 1
        mask[np.where((1.75 < gridXCenter.data < 2.25) &
                      (1.75 < gridYCenter.data < 2.25) &
                      (1.75 < gridZCenter.data < 2.25))] = 0

    # add arbitrary areas values
    if doarea:
        area = grid.add_item(esmpy.GridItem.AREA)
        area[:] = 5.0

    return grid


# TRYING MANUALLY:

In [3]:
lons_out =  np.arange(0, 360, 5)
lats_out = np.arange(-90, 90, 2.0)
alts_out =  np.arange(250, 2000, 50)

ins = xr.open_dataset('../mesh-test/ugrid-mesh-vars.nc')

In [4]:
ins

<xarray.Dataset>
Dimensions:   (nnodes: 1474560, ncells: 1448400, eight: 8, time: 20)
Dimensions without coordinates: nnodes, ncells, eight, time
Data variables:
    nodelon   (nnodes) float64 ...
    nodelat   (nnodes) float64 ...
    height    (nnodes) float64 ...
    vertids   (ncells, eight) int32 ...
    meshtype  (ncells) int32 ...
    edens     (time, nnodes) float64 ...
    odens     (time, nnodes) float64 ...
    ndens     (time, nnodes) float64 ...
    mere      (time, nnodes) float64 ...
    zone      (time, nnodes) float64 ...
    smesh     int32 ...

In [5]:
mesh = esmpy.Mesh(parametric_dim=3, spatial_dim=3, coord_sys=esmpy.CoordSys.SPH_DEG)

In [6]:
mesh.add_nodes(ins.nnodes.max().values+1, 
               np.array([ins.nodelon.values, ins.nodelat.values, ins.height.values]).T,
               ins.vertids.values, np.zeros(ins.nnodes.max().values+1))

In [7]:
fieldin = esmpy.Field(mesh)

In [8]:
fieldin.data[:] = ins.edens.isel(time=0).values

In [9]:
g2 = grid_create_from_coordinates_3d(lats_out, lons_out, alts_out)

In [10]:
fieldout = esmpy.Field(g2)

In [ ]:
regridder = esmpy.Regrid(fieldin, fieldout,
                         regrid_method=esmpy.RegridMethod.BILINEAR,
                         unmapped_action=esmpy.UnmappedAction.ERROR)